In [253]:
%run ../Python_files/load_dicts.py
%run ../Python_files/util.py

In [254]:
import numpy as np

N = np.array([[1, 0, 0, -1, 1, 0], [-1, 1, -1, 1, 0, -1], [0, -1, 1, 0, -1, 1]])

In [255]:
N

array([[ 1,  0,  0, -1,  1,  0],
       [-1,  1, -1,  1,  0, -1],
       [ 0, -1,  1,  0, -1,  1]])

In [256]:
n = 3  # number of nodes
m = 6  # number of links

model = Model("OD_matrix_est_toy_example")

lam = {}
for i in range(n+1)[1:]:
    for j in range(n+1)[1:]:
        if i != j:
            key = str(i) + '->' + str(j)
            lam[key] = model.addVar(name='lam_' + key)
            
x = {}
for k in range(m):
    for i in range(n+1)[1:]:
        for j in range(n+1)[1:]:
            if i != j:
                key = str(k) + '->' + str(i) + '->' + str(j)
                x[key] = model.addVar(name='x_' + key)

model.update() 

In [257]:
x_0 = np.array([100, 100, 50, 200, 150, 200])

In [258]:
x_0

array([100, 100,  50, 200, 150, 200])

In [259]:
# Set objective
obj = 0

for i in range(n+1)[1:]:
    for j in range(n+1)[1:]:
        if i != j:
            key = str(i) + '->' + str(j)
            obj += lam[key] * lam[key]
            
model.setObjective(obj)

In [260]:
# Add constraint: lam >= 0
for i in range(n+1)[1:]:
    for j in range(n+1)[1:]:
        if i != j:
            key = str(i) + '->' + str(j)
            model.addConstr(lam[key] >= 100)
            
for k in range(m):
    s = 0
    for i in range(n+1)[1:]:
        for j in range(n+1)[1:]:
            if i != j:
                key = str(k) + '->' + str(i) + '->' + str(j)
                s += x[key]
                model.addConstr(x[key] >= 0)
    model.addConstr(s - x_0[k] == 0)
    print(s)
            
for l in range(n):
    for i in range(n+1)[1:]:
        for j in range(n+1)[1:]:
            if i != j:
                key_ = str(i) + '->' + str(j)
                s = 0
                for k in range(m):
                    key = str(k) + '->' + str(i) + '->' + str(j)
                    s += N[l, k] * x[key]  
                if (l+1 == i):
                    model.addConstr(s + lam[key_] == 0)
                elif (l+1 == j):
                    model.addConstr(s - lam[key_] == 0)
                else:
                    model.addConstr(s == 0)
                print(s)
                
#                 if (i == 1 and j == 2):
#                     print(s)

model.update()

model.setParam('OutputFlag', False)
model.optimize()

<gurobi.LinExpr: x_0->1->2 + x_0->1->3 + x_0->2->1 + x_0->2->3 + x_0->3->1 + x_0->3->2>
<gurobi.LinExpr: x_1->1->2 + x_1->1->3 + x_1->2->1 + x_1->2->3 + x_1->3->1 + x_1->3->2>
<gurobi.LinExpr: x_2->1->2 + x_2->1->3 + x_2->2->1 + x_2->2->3 + x_2->3->1 + x_2->3->2>
<gurobi.LinExpr: x_3->1->2 + x_3->1->3 + x_3->2->1 + x_3->2->3 + x_3->3->1 + x_3->3->2>
<gurobi.LinExpr: x_4->1->2 + x_4->1->3 + x_4->2->1 + x_4->2->3 + x_4->3->1 + x_4->3->2>
<gurobi.LinExpr: x_5->1->2 + x_5->1->3 + x_5->2->1 + x_5->2->3 + x_5->3->1 + x_5->3->2>
<gurobi.LinExpr: x_0->1->2 + 0.0 x_1->1->2 + 0.0 x_2->1->2 + -1.0 x_3->1->2 + x_4->1->2 + 0.0 x_5->1->2>
<gurobi.LinExpr: x_0->1->3 + 0.0 x_1->1->3 + 0.0 x_2->1->3 + -1.0 x_3->1->3 + x_4->1->3 + 0.0 x_5->1->3>
<gurobi.LinExpr: x_0->2->1 + 0.0 x_1->2->1 + 0.0 x_2->2->1 + -1.0 x_3->2->1 + x_4->2->1 + 0.0 x_5->2->1>
<gurobi.LinExpr: x_0->2->3 + 0.0 x_1->2->3 + 0.0 x_2->2->3 + -1.0 x_3->2->3 + x_4->2->3 + 0.0 x_5->2->3>
<gurobi.LinExpr: x_0->3->1 + 0.0 x_1->3->1 + 0.0 x_2

In [261]:
lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
# print('Obj: %g' % obj.getValue())

# 100, 100, 50, 200, 150, 200

lam_1->2 100
lam_1->3 100
lam_2->1 150
lam_2->3 100
lam_3->1 100
lam_3->2 100
x_0->1->2 0
x_0->1->3 0
x_0->2->1 100
x_0->2->3 0
x_0->3->1 1.48309e-14
x_0->3->2 0
x_1->1->2 1.16631e-14
x_1->1->3 5.96382e-15
x_1->2->1 7.6873e-15
x_1->2->3 5.93659e-15
x_1->3->1 1.91734e-14
x_1->3->2 100
x_2->1->2 6.0494e-15
x_2->1->3 21.4549
x_2->2->1 10.066
x_2->2->3 18.479
x_2->3->1 4.0985e-15
x_2->3->2 5.50592e-15
x_3->1->2 100
x_3->1->3 100
x_3->2->1 0
x_3->2->3 0
x_3->3->1 0
x_3->3->2 0
x_4->1->2 0
x_4->1->3 0
x_4->2->1 50
x_4->2->3 0
x_4->3->1 100
x_4->3->2 0
x_5->1->2 7.64681e-15
x_5->1->3 78.5451
x_5->2->1 39.934
x_5->2->3 81.521
x_5->3->1 4.31022e-15
x_5->3->2 5.30505e-15


In [262]:
sum(lam_list[0:6])

650.0000000002403

In [263]:
sum(x_0)

800